In [18]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import seaborn as sns
from ImportanceImputer import ImportanceImputer

sns.set(
    { "figure.figsize": (17, 7) },
    style='ticks',
    # palette=sns.color_palette("Set2"),
    color_codes=True,
    font_scale=0.8
)
%config InlineBackend.figure_format = 'retina'
import warnings
warnings.filterwarnings('ignore')

# Load data

In [19]:
# Load the dataset
auto = pd.read_csv('adverts.csv')

In [20]:
auto = auto.sample(50000)

In [21]:
imputer = ImportanceImputer()

result = imputer.fit_transform(auto, "standard_colour")

result

Imputing 631 entries.


,standard_colour
87280,Grey
121284,Black
278207,White
45116,Green
115387,Grey
...,...
350941,Silver
90687,Black
374535,Grey
126356,White


In [22]:
auto["standard_colour"] = result["standard_colour"]

In [23]:
auto.loc[result.index]

,public_reference,mileage,reg_code,standard_colour,standard_make,standard_model,vehicle_condition,year_of_registration,price,body_type,crossover_car_and_van,fuel_type
87280,202009294354338,57000.0,62,Grey,Nissan,Qashqai+2,USED,2013.0,6295,SUV,False,Petrol
121284,202010225293115,0.0,NaN,Black,Land Rover,Defender 110,NEW,NaN,60755,SUV,False,Diesel
278207,202008313139271,81000.0,15,White,BMW,3 Series,USED,2015.0,9179,Saloon,False,Diesel
45116,202010054638734,53245.0,07,Green,Honda,Civic,USED,2007.0,1989,Hatchback,False,Petrol
115387,202006059820685,11000.0,68,Grey,Honda,Jazz,USED,2018.0,10495,Hatchback,False,Petrol
...,...,...,...,...,...,...,...,...,...,...,...,...
350941,202006240454083,10.0,NaN,Silver,Hyundai,Santa Fe,NEW,NaN,38075,SUV,False,Diesel
90687,202010094800017,20225.0,17,Black,Renault,Captur,USED,2017.0,9750,SUV,False,Petrol
374535,202009234114526,940.0,20,Grey,Ferrari,GTC4Lusso,USED,2020.0,194900,Coupe,False,Petrol
126356,202010134954452,13375.0,67,White,Toyota,Prius,USED,2017.0,18495,Hatchback,False,Petrol Plug-in Hybrid


In [24]:
# # Filter the DataFrame to include rows where standard_colour is not NaN
# comparison_df = auto[auto['standard_colour'].notna()]

# # Compare the original and imputed columns and calculate accuracy
# matches = (comparison_df['standard_colour'] == comparison_df['standard_colour_imputed']).sum()
# total = len(comparison_df)
# accuracy = matches / total

# print(f"Accuracy: {accuracy:.2%} ({matches}/{total} matches)")

In [25]:
# # Map car colors to line colors
# color_mapping = {
#     "Red": "#F4A6A6",      # Pastel red
#     "Blue": "#A6C8F4",     # Pastel blue
#     "Green": "#A6F4A6",    # Pastel green
#     "White": "#F7F7F7",    # Light gray (near white)
#     "Black": "#000000",    # Light gray for black
#     "Silver": "#CACACA",   # Soft silver
#     "Grey": "#9D9DAD",     # Pastel gray
#     "Orange": "#F7B696",   # Pastel orange
#     'Beige': "#331100",
#     'Yellow': "#FFFF00", 
#     'Gold': "#FFAA00", 
#     'Purple': "#FF00FF", 
#     'Burgundy': "#AA0000", 
#     'Brown': "#441111", 
#     'Bronze': "#441111",
#     'Maroon': "#AA2277",
#     'Multicolour':"#778822",
#     'Pink': "#FFDDDD",
#     'Turquoise': "#BB99FF"
# }

# fig, ax = plt.subplots(figsize=(8,0.5))
# sns.stripplot(auto.loc[auto["price"] < 5e4].sample(500), x="price", jitter=0.3, alpha=0.5,legend=False, ax=ax, hue='standard_colour', dodge=False, palette=color_mapping);